In [ ]:
## @package tf_tutorials
#  This a place where I add any helper notes i learn on tensorflow
#

### Input types in tf
1- A placeholder : you know the type of the variable, the shape but you'll fed the data during execution
(EX: input features).

2- Variable : a tensor with some value that will be updated during execution(EX:weights)

3- constants : a tensor with value that can't be changed

In [36]:
import tensorflow as tf
import numpy as np

In [ ]:
# placeholder example
x = tf.placeholder(tf.float32, (None, 10),name="x") 

In [ ]:
len(x.shape)

In [ ]:
# variable example
w = tf.get_variable("w", shape=(10, 20), dtype=tf.float32)
w = tf.Variable(tf.random_uniform((10, 20)), name="w")

In [ ]:
# constant example
c = tf.constant(np.ones((4, 4)))

### Operations in tensorflow
### Matrix multiplication

In [ ]:
## Matrix multiplication 
z = x @ w
print(z)

### Every node in the graph is an operation
### To list all current operations(nodes) in the default graph use the following comman


In [ ]:
tf.get_default_graph().get_operations()

In [ ]:
### Listint output of the operations in the default graph
tf.get_default_graph().get_operations()[0].outputs

### Running a graph
#### we need a session object that encapsulates the environment in which operations are executed and tensor objects are evaluated
#### tf.Session owns necessary resources to execute your graph, such as tf.Variable, that occupy RAM.
#### It is important to release these resources when they are no longer required with tf.Session.close()

### Example 1

In [ ]:
#####Example
# Definig a graph
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
## Create a session 
s = tf.InteractiveSession()
print(c) # here just looking at the type
print(s.run(c)) # that's how you run the graph


### Note on variables 
# variables are defined and initialized outside the session,
# so we need to get this initialized value inside the graph execution environment
s.run(tf.global_variables_initializer())

##### Example on variables initialization

In [ ]:
# Definig the graph
tf.reset_default_graph()
a = tf.constant(np.ones((2, 2), dtype=np.float32))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b
## running the graph 
s = tf.InteractiveSession()
#s.run(c)  # will create error "Attempting to use uninitialized value Variable"
## the correct way to run
s.run(tf.global_variables_initializer())
s.run(c)
tf.Session.close(s)

### Example on feeding a placeholder

In [ ]:
# Definig the graph
tf.reset_default_graph()
a = tf.placeholder(np.float32, (2, 2))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b
## running the graph 
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())
# s.run(c) ## Incorrect will give error: "you must feed the placeholder"
s.run(c, feed_dict={a: np.ones((2, 2))})
tf.Session.close(s)

### By default all inputs defined as tf.variable are trainable in tf and could be updated in backpropagation.
### you can change that by definig the varaible like this:
### x = tf.get_variable("x", shape=(), dtype=tf.float32, trainable=False)

### Logging with Tensor board:
Tensor board is a visual tool to see the plots of any statistics from the code
To use tensor board :
- Add summary to the graph
    - Add a summary for each variable.
    - merge all these summaries together
- After starting the interactive session:
    - create a summary writer
    - add to summary
    - flush the data to disk to visualize it
After you run the code in the terminal run :
tensorboard --logdir=./logs

### Model checkpoints:
save the variables state with tf.train.saver
and you can restore them with saver.restore(s,"logs/2/model.ckpt-50")
#### Note: only tf.variables will be stored so the graph need to be defined in the same way before storing


### Example on tf optimizers

In [44]:
'''
Create the graph
'''
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2
'''
Create summary for x,f
'''
tf.summary.scalar('curr_x',x)
tf.summary.scalar('curr_f',f)
summaries = tf.summary.merge_all()
##f = tf.Print(f,[x,f],"x,f: ") # output synced value for x,f
'''
minimize f with respect to x using gradient descent
'''
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f, var_list=[x])
### display all trainable variables
tf.trainable_variables()
'''
Start the session
'''
s = tf.InteractiveSession()
'''
creates summary writer
'''
summary_writer = tf.summary.FileWriter("logs/2",s.graph)
s.run(tf.global_variables_initializer())
for i in range(10):
     _,curr_summary= s.run([step,summaries])
     summary_writer.add_summary(curr_summary,i)
     summary_writer.flush()
tf.Session.close(s)
    

### Example on linear regression

In [45]:
'''
Generate a model dataset
'''
N = 1000
D = 3
x = np.random.random((N,D))
w = np.random.random((D,1))
y = x @ w + np.random.randn(N,1)*0.20
print("ground_truth_weights: ", w)

ground_truth_weights:  [[ 0.00969964]
 [ 0.23637935]
 [ 0.99627166]]


In [53]:
'''
Building the graph 
'''
tf.reset_default_graph()
features = tf.placeholder(tf.float32,shape = (None, D))
target = tf.placeholder(tf.float32,shape = (None,1))
weights = tf.get_variable("w",shape=(D,1),dtype=tf.float32)
predictions = features @ weights
# define loss function
loss = tf.reduce_mean((target-predictions)**2)
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)
'''
executing session
'''
s = tf.InteractiveSession()
# model checkpoint
saver = tf.train.Saver(tf.trainable_variables())
s.run(tf.global_variables_initializer())
for i in range(300):
    _,curr_loss,curr_weights = s.run([step,loss,weights],
                                    feed_dict = {features:x,
                                                target:y})
    if i%50 == 0:
        saver.save(s,"logs/2/model.ckpt",global_step=i)
        print(curr_loss)
print(s.run(weights))
tf.Session.close(s)

[[ 0.04280057]
 [ 0.22732031]
 [ 0.98404074]]
[[ 0.04280057]
 [ 0.22732031]
 [ 0.98404074]]


In [47]:
print(tf.__version__)

1.2.1
